In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pycountry

pd.set_option('display.max_rows', None)   # Shows all rows in the DataFrame
pd.set_option('display.max_columns', None)  # Shows all columns in the DataFrame
pd.set_option('display.width', None)     # Ensures no line breaks for wide DataFrames
pd.set_option('display.max_colwidth', None)


# Import Data

In [148]:
df = pd.read_csv('data/SAP Datasets.csv')

# forward and backward filling
years_col = [str(i) for i in range(2022,2024)]
df[years_col] = df[years_col].apply(lambda row: row.ffill().bfill(), axis=1)

# drop unnecessary columns
df = df.drop(['Country Code', 'short description', 'long description', 'Indicator Code'], axis=1)

/var/folders/ds/b8dd7ztn06l6p5dfvy90q01c0000gn/T/ipykernel_12913/964863078.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/SAP Datasets.csv')


# Ranking by Indicators

In [149]:
Indicators = pd.read_csv('data/Indicators_final.csv')

Indicators = Indicators[Indicators['DOES A HIGHER VALUE INCREASE POVERTY?'] != 'TBD']
Indicators

,INDICATOR NAMES,DOES A HIGHER VALUE INCREASE POVERTY?,Short Description,Long Description
0,Control of Corruption: Estimate,FALSE,NaN,NaN
1,Multilateral debt service (% of public and publicly guaranteed debt service),TRUE,"Multilateral debt service is the repayment of principal and interest to the World Bank, regional development banks, and other multilateral agencies.","Multilateral debt service is the repayment of principal and interest to the World Bank, regional development banks, and other multilateral agencies. public and publicly guaranteed debt service is the sum of principal repayments and interest actually paid in currency, goods, or services on long-term obligations of public debtors and long-term private obligations guaranteed by a public entity."
2,"Access to clean fuels and technologies for cooking, rural (% of rural population)",FALSE,NaN,"Access to clean fuels and technologies for cooking, rural is the proportion of rural population primarily using clean cooking fuels and technologies for cooking. Under WHO guidelines, kerosene is excluded from clean cooking fuels."
3,"Access to clean fuels and technologies for cooking, urban (% of urban population)",FALSE,NaN,"Access to clean fuels and technologies for cooking, urban is the proportion of urban population primarily using clean cooking fuels and technologies for cooking. Under WHO guidelines, kerosene is excluded from clean cooking fuels."
4,Access to clean fuels and technologies for cooking (% of population),FALSE,NaN,"Access to clean fuels and technologies for cooking is the proportion of total population primarily using clean cooking fuels and technologies for cooking. Under WHO guidelines, kerosene is excluded from clean cooking fuels."
5,"Access to electricity, rural (% of rural population)",FALSE,NaN,"Access to electricity, rural is the percentage of rural population with access to electricity."
6,"Access to electricity, urban (% of urban population)",FALSE,NaN,"Access to electricity, urban is the percentage of urban population with access to electricity."
7,Access to electricity (% of population),FALSE,NaN,"Access to electricity is the percentage of population with access to electricity. Electrification data are collected from industry, national surveys and international sources."
8,Compensation of employees (current LCU),FALSE,NaN,"Compensation of employees consists of all payments in cash, as well as in kind (such as food and housing), to employees in return for services rendered, and government contributions to social insurance schemes such as social security and pensions that provide benefits to employees."
9,Compensation of employees (% of expense),FALSE,NaN,"Compensation of employees consists of all payments in cash, as well as in kind (such as food and housing), to employees in return for services rendered, and government contributions to social insurance schemes such as social security and pensions that provide benefits to employees."


## Indicator Dataframes

In [150]:
results = {}

for index, row in Indicators.iterrows():
    indicator_name = row['INDICATOR NAMES']
    increase_poverty = row['DOES A HIGHER VALUE INCREASE POVERTY?']
    
    indicator_df = df[df['Indicator Name'] == indicator_name]
    indicator_df.set_index('Country Name', inplace=True)
    
    years_columns = [str(year) for year in range(2000, 2024)]
    country_year_data = indicator_df[years_columns]
    
    if increase_poverty == 'TRUE':
        direction = True  # Lower values get better ranks
    elif increase_poverty == 'FALSE':
        direction = False  # Higher values get better ranks
    
    ranked_countries = country_year_data.rank(axis=0, method='min', ascending=direction, na_option='keep')
    
    column_name = f"{indicator_name}"
    ranked_countries[column_name] = ranked_countries.mean(axis=1, skipna=True)
    
    results[column_name] = ranked_countries[[column_name]].sort_values(by='Country Name').reset_index()

In [151]:
combined_df = pd.concat([df.drop('Country Name', axis=1) for _, df in results.items()], axis=1)
combined_df.insert(0,'Country Name', results['Control of Corruption: Estimate']['Country Name'])

In [152]:
combined_df.head()

,Country Name,Control of Corruption: Estimate,Multilateral debt service (% of public and publicly guaranteed debt service),"Access to clean fuels and technologies for cooking, rural (% of rural population)","Access to clean fuels and technologies for cooking, urban (% of urban population)",Access to clean fuels and technologies for cooking (% of population),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)",Access to electricity (% of population),Compensation of employees (current LCU),Compensation of employees (% of expense),Adjusted savings: education expenditure (current US$),Adjusted net national income per capita (current US$),Terms of trade adjustment (constant LCU),Adequacy of social protection and labor programs (% of total welfare of beneficiary households),Adequacy of unemployment benefits and ALMP (% of total welfare of beneficiary households),Adequacy of social safety net programs (% of total welfare of beneficiary households),Adequacy of social insurance programs (% of total welfare of beneficiary households),Political Stability and Absence of Violence/Terrorism: Estimate,"Literacy rate, youth female (% of females ages 15-24)","Literacy rate, youth (ages 15-24), gender parity index (GPI)","Literacy rate, youth male (% of males ages 15-24)","Literacy rate, youth total (% of people ages 15-24)","Literacy rate, adult female (% of females ages 15 and above)","Literacy rate, adult male (% of males ages 15 and above)","Literacy rate, adult total (% of people ages 15 and above)","Compulsory education, duration (years)","Children out of school, primary","Children out of school, primary, female","Children out of school, female (% of female primary school age)","Children out of school, primary, male","Children out of school, male (% of male primary school age)",Children out of school (% of primary school age),"Adolescents out of school, female (% of female lower secondary school age)","Adolescents out of school, male (% of male lower secondary school age)",Adolescents out of school (% of lower secondary school age),Expenditure on primary education (% of government expenditure on education),Expenditure on secondary education (% of government expenditure on education),Expenditure on tertiary education (% of government expenditure on education),People using at least basic drinking water services (% of population),People using safely managed drinking water services (% of population),"Community health workers (per 1,000 people)",Pregnant women receiving prenatal care (%),People using at least basic sanitation services (% of population),Births attended by skilled health staff (% of total),Current health expenditure (% of GDP),Current health expenditure per capita (current US$),Proportion of people living below 50 percent of median income (%),Multidimensional poverty headcount ratio (UNDP) (% of population),Multidimensional poverty headcount ratio (World Bank) (% of population),"Employers, female (% of female employment) (modeled ILO estimate)","Employers, male (% of male employment) (modeled ILO estimate)","Employers, total (% of total employment) (modeled ILO estimate)","Wage and salaried workers, female (% of female employment) (modeled ILO estimate)","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)","Wage and salaried workers, total (% of total employment) (modeled ILO estimate)","Children in employment, study and work, female (% of female children in employment, ages 7-14)","Children in employment, study and work, male (% of male children in employment, ages 7-14)","Children in employment, study and work (% of children in employment, ages 7-14)","Children in employment, work only, female (% of female children in employment, ages 7-14)","Children in employment, work only, male (% of male children in employment, ages 7-14)","Children in employment, work only (% of children in employment, ages 7-14)","Children in employment, total (% of chil

In [153]:
combined_df['Average_Rank'] = combined_df.iloc[:, 1:].mean(axis=1) 

combined_df_sorted = combined_df.sort_values(by='Average_Rank', ascending=True)

In [154]:
combined_df_sorted[['Country Name', 'Average_Rank']].reset_index()

,index,Country Name,Average_Rank
0,212,Singapore,30.594474
1,235,Switzerland,31.398110
2,40,Cayman Islands,32.090328
3,126,"Korea, Rep.",34.537855
4,173,Netherlands,34.546135
5,13,Australia,34.979301
6,110,Iceland,35.707088
7,153,Malta,36.416077
8,58,Denmark,36.657378
9,196,Portugal,36.753616


In [155]:
combined_df.to_csv('data/country_initial_rankings.csv')

## Normalizing